# ImpDAR  plot_spectrogram() Tutorial

## Introduction

Below are two Matlab files, one from a 2017 data collection season in the South Cascades, Washington state, and the other from a 2019 season at Hercules Dome, Antarctica. We have loaded these files into ImpDAR and converted them to RadarData objects to standardize their attribute names and metadata. After that, we cropped the data to remove the time delay at the top of the radargram. ImpDAR crops the data dynamically to allow you to remove the data collected as a result of the pre-trigger, which is how soon after you start collecting data that the transmitter actually sends out its first pulse. A very common next processing step is to vertically bandbass the data around some central frequency. This central frequency is determined by the type of antenna that is chosen to collect the data with. However, some antennas might not be tuned well, and so there actual central frequency can be very different than that listed for the antenna. The `plot_spectrogram()` method outlined below allows you to visually detect the central frequency that was used to collect the data, taking the guesswork out of the process. We use the [`signal.periodogram()`](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.signal.periodogram.html#scipy.signal.periodogram) method from SciPy which returns an estimate of the power spectral density. This calculates a Fast Fourier Transform and gives us back a histogram of representative frequencies in a signal.

**Note**: the `plot_spectrogram()` assumes that your data is in the frequency domain. A limitation of this is that it cannot be used for data in the power domain, which is unfortunate given some of the benefits of thinking in terms of power.

In [1]:
# We get annoying warnings about backends that are safe to ignore
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

import numpy as np
import scipy
import scipy.signal as signal
import scipy.io as io

import matplotlib.pyplot as plt

from impdar.lib import load
from impdar.lib import process
from impdar.lib import plot


In [2]:
#example files on disk
scg_file = '../data/data03_LINE04_raw.mat'
herc_file = '../data/HDGridE_x53_raw.mat'

#load files
scg_dat = load.load('mat', scg_file)[0]
herc_dat = load.load('mat', herc_file)[0]

Let's do some basic pre-processing:

In [ ]:
#crop the pretrigger from the top of the data
scg_dat.crop(top_or_bottom='top')

In [ ]:
#crop the top of the data
herc_dat.crop(top_or_bottom='top')

#normal move out correction with antenna separation in meters
herc_dat.nmo(ant_sep=171.4)



In [ ]:
#we can look at all of the attributes that a RadarData object in ImpDAR keeps track of
vars(scg_dat)

In [ ]:
#print out the dimensions of each radar gram
print('The SCG data has dimensions {}.'.format(scg_dat.data.shape))
print('The Hercules Dome data has dimensions {}.'.format(herc_dat.data.shape))

ImpDAR's plotting library also makes it easy to visualize radargrams quickly.

In [ ]:
plot.plot_radargram(scg_dat)

In [ ]:
plot.plot_radargram(herc_dat)

The snow data from the South Cascades looks at least recognizable, but often data coming back from Antarctica needs to be bandpassed before if can be visualized in any meaningful way because it is collected using super low frequency systems.

## Plotting Power Spectral Density
As a first example, let's plot the periodogram across traces in the radargram. We can limit the plot's y-limit frequency to 2000 MHz to zoom in on the area of interest.

In [ ]:
plot.plot_spectrogram(scg_dat, freq_limit=(0, 1000))

Command line version of calling this method:

```python
impdar plot -spectra 0 1000 data03_LINE04_raw.mat
```

The central frequency listed for this Pulse Ekko antenna was listed as 500 MHz, but as you can see there is significant ringing at 200 and ~600 MHz. We could run calculate the Fast Fourier Transform using a Hanning Window to try and cut down on the noise a bit.

In [ ]:
plot.plot_spectrogram(scg_dat, freq_limit=(0, 1000), window='hanning')

This shows that the actual central frequency was probably closer to 600 MHz. The danger with using a Hanning window or something similar is that you can lose a lot of important information.

The Hercules Dome data is a little bit different than the South Cascade data in that it had an extra processing step done to it after being cropped but before needing to be bandpassed. We performed a normal moveout correction, about which there is another in-depth Jupyter Notebook tutorial within this repository. The short version of what this correction does is that describes the effect that the distance between a seismic source and a receiver (the offset) has on the arrival time of a reflection in the form of an increase of time with offset. So the NMO correction removes the “V” that we get from the antennae firing and when the signal is received later on as we move, then converts that signal to a fixed point. Note: this is not a full time to distance conversion because we don’t have velocity data.

Here is the shell command that was run:

```python
impproc nmo 171.4 HDGridE_x53_raw.mat
```

Let's plot the Hercules Dome frequency data.

In [ ]:
plot.plot_spectrogram(herc_dat, freq_limit=(0,4))

## Vertically Bandpassing Our Data
This information can be an easy way to identify which central frequencies to bandpass around. If you were working on the command line, you could use the following commands to vertically bandpass the data:

```python
impproc vbp 250 750 data03_LINE04_cropped.mat
```

And:

```python
impproc vbp 1 5 HDGridE_x53_nmo_cropped.mat
```

In the Hercules Dome data, the actual central frequency is 3 MHz, but the low frequency ringing is drowning out the signal. In this case, it helps to have knowledge of the antennae specifications.

Let's go ahead and vertically bandpass filter this data and look at the output.

In [ ]:
#go ahead and use 500 MHz as the central frequency
scg_dat.vertical_band_pass(250, 750)

In [ ]:
#visualize the processed radargram
#this is a more convenient way to call impdar's plotting library with the RadarData object directly
plot.plot_radargram(scg_dat)

The South Cascade data looks a little cleaner, but not muched changed. How about the Hercules Dome data?

In [ ]:
#using 3 MHz as the central frequency
herc_dat.vertical_band_pass(1, 5)

In [ ]:
#visualize the output
plot.plot_radargram(herc_dat)

That made a big difference! Now you can see some of the bedrock topography, and a lot more definition in the layer stratifcation. However, both sets of data could use more processing steps to further clean them up. Next, you could then interpolate, adaptively horizontally filter, and migrate the data. There is no need to migrate South Cascade data, it is mainly for removing the hyperbolae like in the Hercules Dome data. If you want to dig deeper into the migration routines that ImpDAR supports, there is another excellent tutorial within this repository.